In [1]:
import csv
import os
from collections import defaultdict
from datetime import datetime
from datetime import timedelta
from string import punctuation

import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from pandas.tslib import Timestamp

# Preprocesamiento de datos

## Socios activos 2016

In [2]:
# 1.       CTA: categoría para niños y jóvenes hasta bachillerato
# 2.       CTI: categoría infantil, hasta doce años y solo es para Bogotá (blaa –biblioteca Luis Ángel Arango)
# 3.       CTB: público general (solo permite préstamo de libros)
# 4.       CTC: público en general (permite préstamo de libros y material audiovisual)
# 5.       CTF: categoría familiar, pueden acceder a los beneficios hasta tres personas del grupo familiar que se asocia
# 6.       CE1, CE2, CE2: categorías empresariales (solo para instituciones)
# 7.       CIS: categoría Inclusión Social
# 8.       CTU: categoría Universitaria
# 9.       VIP: Investigadores, préstamo de 30 materiales (libros y material audiovisual)

#      Categoría de lector: CE1 Categoría empresarial                                   218
#      Categoría de lector: CE2 Categoría empresarial 2                                   5
#      Categoría de lector: CEM Categoría Empleado/Pensionado                          1593
#      Categoría de lector: CIN Categoría Interbibliotecario                            128
#      Categoría de lector: CIP Categoría Investigador                                  351
#      Categoría de lector: CIS Categoría Inclusión Preferencial                      15974
#      Categoría de lector: CIV Categoría Investigadores                                 72
#      Categoría de lector: CML Categoría Mediador de Proyectos Culturales                2
#      Categoría de lector: CSU Categoría Sucursal                                       91
#      Categoría de lector: CTA Categoría Básica A                                    10610
#      Categoría de lector: CTB Categoría B                                            1359
#      Categoría de lector: CTC Categoría C                                             224
#      Categoría de lector: CTF Categoría familiar                                     1105
#      Categoría de lector: CTI Categoría Infantil                                     1004
#      Categoría de lector: CTU Categoría General                                     17402
#      Categoría de lector: PTE PROCESOS TÉCNICOS                                       95

In [3]:
os.listdir('socios_activos_csv')

['SOCIOS_ACTIVOS_A_18_11_2016.csv']

In [4]:
total_colums_2 = [
    'CODBAR', 'Sucursal', 'Tratamiento', 'Barrio', 'Ciudad', 'CATEG', 'Ocupación', 'Ocupación1',
    'LECOL2', 'Nivel Estudio1', 'Área de Estudio', 'Área de Estudio1',
    'Ult.Uso', 'Fech. Reg', 'Fech. Renov', 'Caduca',
    'Fech. Nacim', 'Fecha _Actual', 'Edad', 'Dev. Retras', 'Mat. Prest', 'Prest. Sala',
    'No. Petic', 'No. Reser_Act', 'No. suspens', 'Ult.uso bibl', 'Prest_año', 'Pres_ult_año'
]

In [5]:
[(x, total_colums_2.index(x)) for x in total_colums_2]

[('CODBAR', 0),
 ('Sucursal', 1),
 ('Tratamiento', 2),
 ('Barrio', 3),
 ('Ciudad', 4),
 ('CATEG', 5),
 ('Ocupación', 6),
 ('Ocupación1', 7),
 ('LECOL2', 8),
 ('Nivel Estudio1', 9),
 ('Área de Estudio', 10),
 ('Área de Estudio1', 11),
 ('Ult.Uso', 12),
 ('Fech. Reg', 13),
 ('Fech. Renov', 14),
 ('Caduca', 15),
 ('Fech. Nacim', 16),
 ('Fecha _Actual', 17),
 ('Edad', 18),
 ('Dev. Retras', 19),
 ('Mat. Prest', 20),
 ('Prest. Sala', 21),
 ('No. Petic', 22),
 ('No. Reser_Act', 23),
 ('No. suspens', 24),
 ('Ult.uso bibl', 25),
 ('Prest_año', 26),
 ('Pres_ult_año', 27)]

In [6]:
%%time

used_colums = [
    # Identificación
    'CODBAR',
    # Localización (I)
    'Sucursal',
    # Género
    'Tratamiento',
    # Localización (II)
    'Barrio', 'Ciudad',
    # Educación/Ocupación
    'CATEG', 'Ocupación1', 'Nivel Estudio1', 'Área de Estudio1',
    # Edad
    'Fech. Nacim',
]

ls = []
for filename in os.listdir('socios_activos_csv'):
    df = pd.read_csv('socios_activos_csv/' + filename, header=0, usecols=used_colums)
    ls.append(df)
perfiles_df = pd.concat(ls, ignore_index=True)

CPU times: user 236 ms, sys: 8 ms, total: 244 ms
Wall time: 243 ms


In [7]:
perfiles_df.shape

(50274, 10)

In [8]:
perfiles_df.head()

,CODBAR,Sucursal,Tratamiento,Barrio,Ciudad,CATEG,Ocupación1,Nivel Estudio1,Área de Estudio1,Fech. Nacim
0,88180782.0,BLAA,Vacío,Chapinero,BOGOTA/COLOMBIA,CIN,Universidad,Err:502,Err:502,NaN
1,88183608.0,BLAA,SRA.,AYACUYO II,BOGOTA/COLOMBIA,CEM,Pensionado,Err:502,Err:502,30/07/57
2,88181732.0,BLAA,SRA.,Icata,BOGOTA/COLOMBIA,CEM,Empleado,Profesional\n,Filosofía y letras,NaN
3,88182347.0,ARM,SR.,MONTENEGRO,MONTENEGRO/COLOMBIA,CEM,Empleado,Profesional\n,Economía,20/09/63
4,88181284.0,BLAA,SR.,LA SOLEDAD,BOGOTA/COLOMBIA,CEM,Err:502,Err:502,Err:502,NaN


#### CODBAR

In [9]:
%%time

def process_codbar(x):
    try:
        r = str(int(x['CODBAR']))
    except:
        r = x['CODBAR']
    return r

perfiles_df['CODBAR'] = perfiles_df.apply(process_codbar, axis=1)

CPU times: user 1.4 s, sys: 16 ms, total: 1.42 s
Wall time: 1.42 s


#### Sucursal

In [10]:
%%time

def process_sucursal(x):
    try:
        r = x['Sucursal'].strip()
    except:
        r = x['Sucursal']
    return r

perfiles_df['Sucursal'] = perfiles_df.apply(process_sucursal, axis=1)

CPU times: user 1.24 s, sys: 8 ms, total: 1.24 s
Wall time: 1.25 s


#### Tratamiento

In [11]:
%%time

def process_tratamiento(x):
    try:
        r = x['Tratamiento'].strip().lower()
        if r == 'vacío':
            r = ''
    except:
        r = x['Tratamiento']
    return r

perfiles_df['Tratamiento'] = perfiles_df.apply(process_tratamiento, axis=1)

CPU times: user 1.22 s, sys: 4 ms, total: 1.22 s
Wall time: 1.22 s


#### Barrio

In [12]:
%%time

def process_barrio(x):
    try:
        r = x['Barrio'].strip().lower()
    except:
        r = x['Barrio']
    return r

perfiles_df['Barrio'] = perfiles_df.apply(process_barrio, axis=1)

CPU times: user 1.21 s, sys: 4 ms, total: 1.21 s
Wall time: 1.21 s


#### Ciudad

In [13]:
%%time

def process_ciudad(x):
    try:
        r = x['Ciudad'].strip().lower()
        r = r.replace('á', 'a').replace('é', 'e').replace('í', 'i').replace('ó', 'o').replace('ú', 'u')
        for p in punctuation:
            r = r.replace(p, ' ')
        r.strip()
        rl = r.split()
        while len(rl) > 1 and rl[-1] in ['colombia','col','amazonas','isla','islas','isla,','islas,','cundinamarca']:
            r = ' '.join(rl[:-1]).strip()
            rl = r.split()
        r.strip()
    except:
        r = x['Ciudad']
    return r

perfiles_df['Ciudad'] = perfiles_df.apply(process_ciudad, axis=1)

CPU times: user 1.75 s, sys: 12 ms, total: 1.76 s
Wall time: 1.76 s


#### CATEG

In [14]:
%%time

def process_categ(x):
    try:
        r = x['CATEG'].strip()
    except:
        r = x['CATEG']
    return r

perfiles_df['CATEG'] = perfiles_df.apply(process_categ, axis=1)

CPU times: user 1.25 s, sys: 20 ms, total: 1.27 s
Wall time: 1.27 s


#### Ocupación1

In [15]:
%%time

def process_ocupacion(x):
    try:
        r = x['Ocupación1'].strip().lower()
        if r == 'err:502':
            r = ''
    except:
        r = x['Ocupación1']
    return r

perfiles_df['Ocupación1'] = perfiles_df.apply(process_ocupacion, axis=1)

CPU times: user 1.22 s, sys: 12 ms, total: 1.23 s
Wall time: 1.23 s


#### Nivel estudio1

In [16]:
%%time

def process_nivel_estudio(x):
    try:
        r = x['Nivel Estudio1'].strip().lower()
        if r == 'err:502':
            r = ''
    except:
        r = x['Nivel Estudio1']
    return r

perfiles_df['Nivel Estudio1'] = perfiles_df.apply(process_nivel_estudio, axis=1)

CPU times: user 1.23 s, sys: 0 ns, total: 1.23 s
Wall time: 1.23 s


#### Área de estudio1

In [17]:
%%time

def process_area_estudio(x):
    try:
        r = x['Área de Estudio1'].strip().lower()
        if r == 'err:502':
            r = ''
    except:
        r = x['Área de Estudio1']
    return r

perfiles_df['Área de Estudio1'] = perfiles_df.apply(process_area_estudio, axis=1)

CPU times: user 1.27 s, sys: 4 ms, total: 1.27 s
Wall time: 1.27 s


#### Fech. Nacim

In [18]:
%%time

def process_fecha_nacimiento(x):
    try:
        day_str, month_str, year_str = x['Fech. Nacim'].split('/')
        if int(year_str) < 16:
            year_str = '20' + year_str
        else:
            year_str = '19' + year_str
        r = pd.to_datetime('/'.join([day_str, month_str, year_str]))
    except:
        r = pd.to_datetime(x['Fech. Nacim'])
    return r

perfiles_df['Fech. Nacim'] = perfiles_df.apply(process_fecha_nacimiento, axis=1)

CPU times: user 15 s, sys: 100 ms, total: 15.1 s
Wall time: 15.1 s


#### Edad

In [19]:
%%time

def calculate_edad(x):
    return 2016 - x['Fech. Nacim'].year

perfiles_df['Edad'] = perfiles_df.apply(calculate_edad, axis=1)

CPU times: user 5.93 s, sys: 12 ms, total: 5.94 s
Wall time: 5.95 s


In [20]:
perfiles_df.to_csv('socios_activos_2016.csv', index=False)